In [1]:
import tensorflow as tf
from tensorflow.contrib import predictor

export_dir = "/Users/sebastianjaszczur/PycharmProjects/deepsat/attention_cmp/1552640099"
predict_fn = predictor.from_saved_model(export_dir)

INFO:tensorflow:Restoring parameters from /Users/sebastianjaszczur/PycharmProjects/deepsat/attention_cmp/1552640099/variables/variables


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import random

In [3]:
from time import time

In [4]:
import sys 
sys.path.insert(0,'../..')

In [5]:
from cnf_dataset import clauses_to_matrix
from dpll import DPLL, RandomClauseDPLL, MostCommonVarDPLL, RandomVarDPLL, JeroslowWangDPLL
import cdcl
CDCL = cdcl.CDCL
from cnf import get_random_kcnf, CNF, get_sats_SR, get_pos_SR, get_sat_SR, get_random_sat_kcnf
from tqdm import tqdm
from collections import Counter

In [6]:
LIMIT_RUNS = 1000

In [7]:
CDCL().run(CNF([[-1, -2], [+1, -2], [+1, +2]]))

[1, -2]

In [8]:
JeroslowWangDPLL().run(CNF([[-1, -2], [+1, -2], [+1, +2]]))

[-2, 1]

In [9]:
cnf = get_pos_SR(15, 15, 2000)
print("Generated")
# print(cnf)
start = time()
CDCL().run(cnf)
end = time()
print(end-start)
start = time()
DPLL().run(cnf)
end = time()
print(end-start)

Generated
0.013300180435180664
0.026754140853881836


In [10]:
import math
from collections import defaultdict

def jw(clauses):
    score = defaultdict(int)

    for clause in clauses:
        for l in clause:
            score[l] += 2. ** (-len(clause))

    return max(score, key=score.get)

In [11]:
jw([[-1, -2], [+1, -2]])

-2

In [12]:
np.set_printoptions(precision=3, suppress=True)

In [13]:
import tensorflow as tf
import os

BATCH_SIZE = 1

In [14]:
VERBOSE = False
TIMEOUT = 10000
cdcl.VERBOSE = VERBOSE
cdcl.TIMEOUT = TIMEOUT

class GraphBasedCDCL(CDCL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        output = predict_fn({"input": inputs})
        policy_probs = output['policy_probabilities']
        sat_prob = output['sat_probabilities'][0]
        
        #print("CNF:", input_cnf, end=' ', file=logfile)
        #print('cnf_clauses[0]', input_cnf.clauses, end=' ', file=logfile)
        #print("probs:\n",  policy_probs[0], end='\n', file=logfile)
        
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                #print(svar, svar_prob, best_prob, file=logfile)
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        #print("best_svar:", best_svar, file=logfile)
        if VERBOSE:
            print("Chosen neural", best_svar)
            print("Pred SAT prob", sat_prob)
        return best_svar

class HumbleGraphBasedCDCL(CDCL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        output = predict_fn({"input": inputs})
        policy_probs = output['policy_probabilities']
        sat_prob = output['sat_probabilities'][0]
        
        #print("CNF:", input_cnf, end=' ', file=logfile)
        #print('cnf_clauses[0]', input_cnf.clauses, end=' ', file=logfile)
        #print("probs:\n",  policy_probs[0], end='\n', file=logfile)
        
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                #print(svar, svar_prob, best_prob, file=logfile)
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        #print("best_svar:", best_svar, file=logfile)
        if VERBOSE:
            print("Chosen neural", best_svar)
            print("Pred SAT prob", sat_prob)
        if sat_prob < 0.3:
            # Overwriting with JW
            best_svar = jw(input_cnf.clauses)
            if VERBOSE:
                print("Choosing JW", best_svar)
        #if sat_prob < 0.02:
        #    print("Aborting, prob too small")
        #    raise ValueError("Prob too small")
        return best_svar

class JeroslawCDCL(CDCL):
    def suggest(self, cnf: CNF):
        res = jw(cnf.clauses)
        if VERBOSE:
            print("Chosen JW", res)
        return res

In [15]:
HumbleGraphBasedCDCL().run(CNF([[-1, -2], [+1, +3], [+2, -3], [+3, -2], [+1, -2], [2, 3]]))

In [16]:
def compute_steps(sats, cdcl_cls):
    steps = []
    solved = 0
    for sat in sats:
        cdcl = cdcl_cls()
        res = cdcl.run(sat)
        # assert res is not None
        if res is not None:
            steps.append(cdcl.number_of_runs)
            solved += 1
    print("Within {} steps solved {} problems out of {}".format(LIMIT_RUNS, solved, len(sats)))
    return steps

In [17]:
def compute_and_print_steps(sats, dpll_cls):
    try:
        print("")
        print("Results of {}".format(dpll_cls.__name__))
        steps = compute_steps(sats, dpll_cls)
        print("#Sats: {}; avg step: {:.2f}; stdev step: {:.2f}".format(
            len(steps), np.mean(steps), np.std(steps)))
        print("Table: {}".format(steps))
    except:
        print("Timeout!", TIMEOUT, "steps")
    """
    plt.figure(figsize=(14, 4))
    plt.subplot(1, 2, 1)
    plt.title("Steps of {}".format(dpll_cls.__name__))
    plt.hist(steps, bins=20) # range(2**(N+1)))
    plt.ylim((0, len(sats)))
    ""
    plt.subplot(1, 2, 2)
    plt.title("Errors of {}".format(dpll_cls.__name__))
    plt.hist(errors, bins=range(N+1))
    plt.ylim((0, len(sats)))
    plt.show()
    """

In [18]:
def print_all(s, n, m, light=False, seed=1, to_test=[HumbleGraphBasedCDCL, GraphBasedCDCL, JeroslawCDCL]):
    print("Starting...")
    # def get_sats_SR(sample_number, min_variable_number, clause_number, max_variable_number=None):
    global S, N, M
    S = s
    N = n # number of clauses
    M = m # number of variables
    
    MAX_TRIES = 100000
    sats = []
    
    random.seed(seed)
    np.random.seed(seed)
    
    for index in range(MAX_TRIES):
        if len(sats) >= S:
            break
        sat = get_pos_SR(M, M, N)
        # if DPLL().run(sat) is not None:
        sats.append(sat)
    assert len(sats) == S
    # sats = get_sats_SR(S,M,N)
    # for sat in sats:
    #    print(sat)
    # assert len(sats) == S
    print("We have generated {} formulas".format(len(sats)))
    # compute_and_print_steps(sats, DPLL)
    # compute_and_print_steps(sats, SimplifiedGraphBasedDPLL)
    
    
    # compute_and_print_steps(sats, SimplifiedMostCommonDPLL)
    # compute_and_print_steps(sats, NormalizedMostCommonDPLL)
    
    #compute_and_print_steps(sats, GraphBasedCDCL)
    for method in to_test:
        compute_and_print_steps(sats, method)
    #logfile.flush()

In [19]:
#print_all(10, 200, 50)

In [21]:
print_all(10, 1000, 40)

Starting...
We have generated 10 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 21.10; stdev step: 16.74
Table: [26, 12, 13, 14, 13, 70, 14, 15, 18, 16]

Results of GraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 25.80; stdev step: 35.47
Table: [10, 12, 13, 14, 13, 132, 14, 15, 19, 16]

Results of JeroslawCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 42.00; stdev step: 20.39
Table: [54, 78, 57, 11, 11, 35, 59, 44, 27, 44]


In [22]:
print_all(10, 1000, 50)

Starting...
We have generated 10 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 34.60; stdev step: 38.25
Table: [11, 8, 25, 24, 16, 16, 68, 20, 139, 19]

Results of GraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 64.90; stdev step: 99.87
Table: [11, 58, 27, 24, 16, 102, 354, 20, 18, 19]

Results of JeroslawCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 57.50; stdev step: 39.37
Table: [29, 12, 108, 47, 60, 125, 65, 14, 99, 16]


In [18]:
print_all(1, 2000, 40, seed=4)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 12.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 76.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 15.00; stdev step: 0.00


In [23]:
print_all(30, 200, 20)

Starting...
We have generated 30 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 30 problems out of 30
#Sats: 30; avg step: 10.57; stdev step: 5.89

Results of GraphBasedCDCL
Within 1000 steps solved 30 problems out of 30
#Sats: 30; avg step: 10.70; stdev step: 7.01

Results of JeroslawCDCL
Within 1000 steps solved 30 problems out of 30
#Sats: 30; avg step: 10.63; stdev step: 5.06


In [36]:
print_all(10, 2000, 30)

Starting...
We have generated 10 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 13.10; stdev step: 8.93

Results of GraphBasedCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 12.90; stdev step: 9.49

Results of JeroslawCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 19.10; stdev step: 11.06


In [ ]:
print_all(10, 2000, 40)

In [19]:
print_all(1, 10000, 50)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 11.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 11.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 29.00; stdev step: 0.00


In [20]:
print_all(1, 10000, 60)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 15.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 15.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 136.00; stdev step: 0.00


In [21]:
print_all(1, 10000, 70)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 19.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 19.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 353.00; stdev step: 0.00


In [22]:
print_all(1, 10000, 80)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 25.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 302.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 419.00; stdev step: 0.00


In [41]:
print_all(1, 10000, 100, seed=2, to_test=[HumbleGraphBasedCDCL])

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 978
to-date set lits: []
Chosen neural -29
Pred SAT prob 0.44459802
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 948
to-date set lits: [-29]
Chosen neural -54
Pred SAT prob 0.2593391
Choosing JW 61
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 916
to-date set lits: [-29, 61]
Chosen neural -34
Pred SAT prob 0.14293711
Choosing JW -9
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 893
to-date set lits: [-9, -29, 61]
Chosen neural 56
Pred SAT prob 0.55247706
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 867
to-date set lits: [-9, -29, 56, 61]
Chosen neural 59
Pred SAT prob 0.4474415
--- DEBUG
original clauses numb 978
learnt clauses numb 0
current clauses numb 842
to-date set lits: [-9, -29, 56, 59, 61]
Chosen

In [23]:
print_all(1, 10000, 90)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 24.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 24.00; stdev step: 0.00

Results of JeroslawCDCL
Timeout! 1000 steps


In [24]:
print_all(1, 100000, 100)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 42.00; stdev step: 0.00

Results of GraphBasedCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 42.00; stdev step: 0.00

Results of JeroslawCDCL
Within 1000 steps solved 1 problems out of 1
#Sats: 1; avg step: 201.00; stdev step: 0.00


In [25]:
print_all(1, 100000, 110)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Timeout! 1000 steps

Results of GraphBasedCDCL
Timeout! 1000 steps

Results of JeroslawCDCL
Timeout! 1000 steps


In [26]:
print_all(1, 100000, 120)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Timeout! 1000 steps

Results of GraphBasedCDCL
Timeout! 1000 steps

Results of JeroslawCDCL
Timeout! 1000 steps


In [27]:
print_all(1, 100000, 130)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Timeout! 1000 steps

Results of GraphBasedCDCL
Timeout! 1000 steps

Results of JeroslawCDCL
Timeout! 1000 steps


In [28]:
print_all(1, 100000, 140)

Starting...


error: Caught keyboard interrupt

In [ ]:
print_all(1, 100000, 150)

In [31]:
print_all(1, 100000, 150, seed=2)

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Timeout! 100 steps

Results of GraphBasedCDCL
Timeout! 100 steps

Results of JeroslawCDCL
Timeout! 100 steps


In [ ]:
print_all(100, 200, 10)

In [ ]:
print_all(10, 500, 25, light=True)

In [ ]:
print_all(10, 500, 30, light=True)

In [ ]:
print_all(10, 1000, 40, light=True)

In [ ]:
print_all(10, 1000, 50, light=True)

In [ ]:
print_all(10, 1000, 70, light=True)

In [ ]:
print_all(10, 1000, 90, light=True)

In [58]:
print(3)

3


In [ ]:
cnf, solution = get_random_sat_kcnf(3, 1000, 5000)
print("generated")
HumbleGraphBasedCDCL().run(cnf)

generated


In [85]:
print_all(100, 100000, 10)

Starting...
We have generated 100 formulas

Results of HumbleGraphBasedCDCL
Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 4.41; stdev step: 1.34

Results of GraphBasedCDCL
Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 4.55; stdev step: 1.50

Results of JeroslawCDCL
Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 5.01; stdev step: 1.82


In [59]:
for s in range(10, 40):
    print_all(1, 10000, 180, seed=s, to_test=[HumbleGraphBasedCDCL])

Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Aborting, prob too small
Timeout! 10000 steps
Starting...
We have generated 1 formulas

Results of HumbleGraphBasedCDCL
Aborting, prob too small
Timeout! 10000 steps
Starting...


error: Caught keyboard interrupt